In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git # installing latest version of transformers library
!pip install torch # torch
!pip install peft # necessary for finetuning of the large model via LoRA approach
!pip install bitsandbytes # necessary for quantiziation
!pip install evaluate # extension of the transformers library
!pip install datasets # extension of the transformers library
!pip install accelerate

In [ ]:
"""
Minimalistic example of finetuning Gemma for sentiment classification
using PEFT and bitsandbytes for memory-efficient training
"""

import torch
from datasets import load_dataset
from transformers import (
    GemmaForSequenceClassification,
    GemmaTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb
from sklearn.metrics import accuracy_score, f1_score

# 1. Load the model with quantization
model_id = "google/gemma-2-2b-it"  # Can also use gemma-7b if you have more resources

# Setup quantization config for loading in 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization config
model = GemmaForSequenceClassification.from_pretrained(
    model_id,
    num_labels=3,  # Binary sentiment classification (positive/negative)
    quantization_config=bnb_config,
    device_map="auto",token=''
)

# 2. Prepare model for PEFT
model = prepare_model_for_kbit_training(model)

# 3. Set up LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",  # Sequence classification task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# 4. Load tokenizer
tokenizer = GemmaTokenizer.from_pretrained(model_id,token='')

# 5. Load dataset (e.g., IMDb for sentiment analysis)
dataset = load_dataset("tweet_eval", "sentiment")
id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}


# Simulate the few-shot regime by sampling 8 examples per class
train_dataset = dataset["train"].shuffle().select(range(100))
eval_dataset = dataset["validation"].select(range(100))

# Create a prompt prefix for sentiment classification
# This helps guide the model to understand the task
sentiment_prompt_prefix = """
Classify the sentiment of the following tweet as negative, neutral, or positive.

Tweet: {text}

Sentiment:
"""

# Function to apply the prompt prefix to the dataset
def add_prompt_prefix(example):
    example["text"] = sentiment_prompt_prefix.format(text=example["text"])
    return example

# Apply the prompt prefix to both training and evaluation datasets
train_dataset = train_dataset.map(add_prompt_prefix)
eval_dataset = eval_dataset.map(add_prompt_prefix)

# Display an example to verify the prompt formatting
print("Example with prompt prefix:")
print(train_dataset[0]["text"])
print(f"Label: {id2label[train_dataset[0]['label']]}")


# 6. Preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

# 7. Create a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
    }

# 9. Define training arguments
training_args = TrainingArguments(
    output_dir="./gemma-sentiment",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",  # Disable wandb/tensorboard if not needed
)

# 10. Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [ ]:
# 11. Train the model
trainer.train()

# 12. Save the trained model
model.save_pretrained("./gemma-sentiment-peft")
tokenizer.save_pretrained("./gemma-sentiment-peft")

# 13. Example inference with finetuned model
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return "Positive" if predicted_class == 1 else "Negative"

# Test with example text
example = "I really enjoyed this movie, the acting was superb!"
print(f"Text: {example}")
print(f"Predicted sentiment: {predict_sentiment(example)}")

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
eval_results